In [5]:
!jupyter nbconvert --to=python AutoEncoder.ipynb

[NbConvertApp] Converting notebook AutoEncoder.ipynb to python
[NbConvertApp] Writing 1705 bytes to AutoEncoder.py


In [1]:
from keras import Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Dense
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# load dataset
# MNIST, for now
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_width, img_height, img_depth = x_train.shape[1], x_train.shape[2], 1

In [3]:
# Data preparation
x_train = x_train/256
x_test = x_test/256
x_train = x_train.reshape(len(x_train), img_width, img_height, img_depth)
x_test = x_test.reshape(len(x_test), img_width, img_height, img_depth)

In [4]:
input_img = Input(
    shape=(x_train.shape[1], x_train.shape[2], 1)
)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='relu', padding='same')(x)

W0715 14:42:23.597991 47352237524032 deprecation_wrapper.py:119] From /cluster/home/akozharin/python_envs/keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 14:42:23.610045 47352237524032 deprecation_wrapper.py:119] From /cluster/home/akozharin/python_envs/keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 14:42:23.612736 47352237524032 deprecation_wrapper.py:119] From /cluster/home/akozharin/python_envs/keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 14:42:23.625343 47352237524032 deprecation_wrapper.py:119] From /cluster/home/akozharin/python_envs/keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.

In [5]:
autoencoder = Model(input_img, decoded)

In [7]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

W0715 14:42:23.695525 47352237524032 deprecation_wrapper.py:119] From /cluster/home/akozharin/python_envs/keras/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0715 14:42:23.707782 47352237524032 deprecation_wrapper.py:119] From /cluster/home/akozharin/python_envs/keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0715 14:42:23.711881 47352237524032 deprecation.py:323] From /cluster/home/akozharin/python_envs/keras/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
NUM_EPOCHS = 100
autoencoder.fit(
    x_train, x_train, 
    epochs=NUM_EPOCHS, 
    batch_size=128, 
    shuffle=True, 
    validation_data=(x_test, x_test)
)

W0715 14:42:23.921315 47352237524032 deprecation_wrapper.py:119] From /cluster/home/akozharin/python_envs/keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 28s 459us/step - loss: 0.1749 - val_loss: 0.1278
Epoch 2/10
60000/60000 [==============================] - 27s 450us/step - loss: 0.1270 - val_loss: 0.1224
Epoch 3/10
60000/60000 [==============================] - 27s 454us/step - loss: 0.1168 - val_loss: 0.1079
Epoch 4/10
60000/60000 [==============================] - 27s 453us/step - loss: 0.1097 - val_loss: 0.1050
Epoch 5/10
60000/60000 [==============================] - 27s 451us/step - loss: 0.1065 - val_loss: 0.1008
Epoch 6/10
60000/60000 [==============================] - 27s 454us/step - loss: 0.1034 - val_loss: 0.0977
Epoch 7/10
60000/60000 [==============================] - 27s 451us/step - loss: 0.1010 - val_loss: 0.0972
Epoch 8/10
60000/60000 [==============================] - 27s 451us/step - loss: 0.1008 - val_loss: 0.0965
Epoch 9/10
60000/60000 [==============================] - 27s 453us/step - loss: 0.0985 - val_

## Save the model

In [12]:
FILENAME = 'example_save.h5'
autoencoder.save(FILENAME)